# DIVAnd in generalized vertical coordinates
## Import packages

In [7]:
using NCDatasets
using DataStructures
using DIVAnd
using Dates
using Statistics
using Random
using Makie
using CairoMakie
using GeoMakie
using Interpolations
using Roots
include("../config.jl")

LoadError: failed to find source of parent package: "Pkg"

## Configuration
* Define the horizontal, vertical (depth levels) and temporal resolutions.
* Select the variable of interest

In [2]:
dx, dy = 0.125, 0.125
lonr = 11.5:dx:20
latr = 39:dy:46

depthr = [0.,5., 10., 15., 20., 25., 30., 40., 50., 66, 
    75, 85, 100, 112, 125, 135, 150, 175, 200, 225, 250, 
    275, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 
    800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 
    1300, 1350, 1400, 1450, 1500, 1600, 1750, 1850, 2000];
#depthr = depthr[1:30];

## Read your ODV file
Adapt the `datadir` and `datafile` values.     
The example is based on a sub-setting of the [Mediterranean Sea](https://www.seadatanet.org/Products#/metadata/cd552057-b604-4004-b838-a4f73cc98fcf) aggregated dataset.      
The dataset has been extracted around the Adriatic Sea and exported to a netCDF using [Ocean Data View](https://odv.awi.de/).
<img src="./Images/MedSeaAggreg.jpg" width="450px">

Download the data files (test and full) if needed.

In [3]:
datafile = adriaticfile
download_check(datafile, adriaticfileURL)

[ Info: File already downloaded


Read the full file:

In [4]:
@time obsval,obslon,obslat,obsdepth,obstime,obsid = NCODV.load(Float64, datafile, 
    "Water body salinity");

  6.329560 seconds (13.86 M allocations: 947.536 MiB, 7.16% gc time, 74.28% compilation time)


In [5]:
f = GeoMakie.Figure()
ax = GeoAxis(f[1,1], dest = "+proj=merc", title="Observations")
GeoMakie.scatter!(ax, obslon, obslat; markersize=2, color = :black)
f

LoadError: UndefVarError: `GeoMakie` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

Check the extremal values of the observations

In [ ]:
checkobs((obslon,obslat,obsdepth,obstime),obsval,obsid)

## 2. Extract the bathymetry
It is used to delimit the domain where the interpolation is performed.

### 2.1 Choice of bathymetry
Modify `bathname` according to the resolution required.

In [ ]:
bathname = gebco08file
download_check(bathname, gebco08fileURL)

In [ ]:
@time bx,by,b = load_bath(bathname,true,lonr,latr);

In [ ]:
figure("Adriatic-Bathymetry")
ax = subplot(1,1,1)
pcolor(bx, by, permutedims(b, [2,1]));
colorbar(orientation="vertical", shrink=0.8).ax.tick_params(labelsize=8)
contour(bx, by, permutedims(b, [2,1]), [0, 0.1], colors="k", linewidths=.5)
gca().set_aspect(aspectratio)
ax.tick_params("both",labelsize=6)

### 2.2 Create mask
* False for sea        
* True for land

In [ ]:
mask = falses(size(b,1),size(b,2),length(depthr))
for k = 1:length(depthr)
    for j = 1:size(b,2)
        for i = 1:size(b,1)
            mask[i,j,k] = b[i,j] >= depthr[k]
        end
    end
end
@show size(mask)

In [ ]:
figure("Adriatic-Mask")
ax = subplot(1,1,1)
gca().set_aspect(aspectratio)
ax.tick_params("both",labelsize=6)
ax.pcolor(bx, by, transpose(mask[:,:,1]), cmap=PyPlot.cm.binary_r)
ax.set_title("Land-sea mask")

### 2.3 Edit the mask
As an example we will remove the Tyrrhenian Sea from the domain.

In [ ]:
grid_bx = [i for i in bx, j in by];
grid_by = [j for i in bx, j in by];

In [ ]:
mask_edit = copy(mask);
sel_mask1 = (grid_by .<= 42.6) .& (grid_bx .<= 14.);
sel_mask2 = (grid_by .<= 41.2) .& (grid_bx .<= 16.2);
mask_edit = mask_edit .* .!sel_mask1 .* .!sel_mask2;
@show size(mask_edit)

In [ ]:
#sel = (obslon .<  maximum(bx)) .& (obslon .>  minimum(bx)) .& (obslat .<  maximum(by)) .& (obslat .>  minimum(bx))
#obslon=obslon[sel]
#obslat=obslat[sel]
#obsdepth=obsdepth[sel]
#obsval=obsval[sel]

The edited mask now looks like this:

In [ ]:
figure("Adriatic-Mask-Edited")
ax = subplot(1,1,1)
ax.tick_params("both",labelsize=6)
ax.pcolor(bx, by, mask_edit[:,:,1]', cmap=cmap=PyPlot.cm.binary_r)
ax.set_title("Edited land-sea mask")
gca().set_aspect(aspectratio)

## Define a vertical correlation length function in real space

In [ ]:
function Lvert(x,y,z)
    return 5.0 + z/5.0
end

LHOR=.5

eps2=0.01

iscyclic=falses(3)

## 3. For comparison, analysis in Cartesian Coordinates (degrees for simplicity)

In [ ]:
xc,yc,zc=ndgrid(lonr,latr,depthr)
pmc = ones(size(xc)) / (xc[2,1,1]-xc[1,1,1]);
pnc = ones(size(xc)) / (yc[1,2,1]-yc[1,1,1]);
poc = ones(size(xc)) / (zc[1,1,2]-zc[1,1,1]);


In [ ]:
sz = (length(lonr),length(latr),length(depthr));
lenx = fill(LHOR,sz)  
leny = lenx./aspectratio
lenz = Lvert.(xc,yc,zc);
@show mean(lenx),mean(leny),mean(lenz)
epsilon2 = eps2;
#epsilon2 = epsilon2 * rdiag;

In [ ]:
obsval=obsdepth
obsmean=mean(obsval)

obslon=[18.]
obslat=[41.375]
obsdepth=[80.]
obsval=[1.]
obsmean=0.

In [ ]:
fi,s=DIVAndrun(mask_edit,(pmc,pnc,poc),(xc,yc,zc),(obslon,obslat,obsdepth),obsval.-obsmean,(lenx,leny,lenz),epsilon2);

In [ ]:
jcut=20
pcolor(xc[:,jcut,:],-zc[:,jcut,:],fi[:,jcut,:].+obsmean,shading="nearest"),colorbar()
plot(bx,-b[:,jcut])
xlim(16.5,19.5)
ylim(-1200,50)

In [ ]:
pcolor(xc[:,:,1],yc[:,:,1],fi[:,:,1].+obsmean,shading="nearest"),colorbar()
gca().set_aspect(aspectratio)

In [ ]:
kcut=10
pcolor(xc[:,:,kcut],yc[:,:,kcut],fi[:,:,kcut].+obsmean,shading="nearest"),colorbar()
gca().set_aspect(aspectratio)

In [ ]:
@show zc[1,1,kcut],yc[1,jcut,1]

# 4.  Coordinate change

## First define sigma coordinates (from 0 to 1)

In [ ]:
Nsigma=20
xs,ys,sigma=ndgrid(lonr,latr,range(0;stop=1,length=Nsigma))
pms = ones(size(xs)) / (xs[2,1,1]-xs[1,1,1]);
pns = ones(size(xs)) / (ys[1,2,1]-ys[1,1,1]);
pos = ones(size(xs)) / (sigma[1,1,2]-sigma[1,1,1]);
sz=size(xs)
masks=trues(size(xs))
masks[:,:,:].=mask_edit[:,:,1];

In [ ]:
jcut=20

In [ ]:
# A way to create a fake bottomgraphy just to guide the vertical grid generation
# Take the original one with the desired mask, then filter and finally make it deep in the shallow regions (to avoid
# to strong changes of z lines)
bfake=b.*mask_edit[:,:,1]
bfake2=DIVAnd.diffusion(trues(size(mask_edit[:,:,1])),(pms[:,:,1],pns[:,:,1]),(.09,.09),bfake)
bmax=maximum(bfake2)
bfake3=0.5.*bfake2 .+ 0.5.*(bfake2./bmax).^0.3*bmax

plot(bx,-bfake2[:,jcut])
plot(bx,-bfake[:,jcut])
plot(bx,-bfake3[:,jcut])

In [ ]:
# Now you can chose to create the vertical coordinate using the "exact" topography or the fake on

#bfun=linear_interpolation((bx,by),b.*masks[:,:,1];extrapolation_bc=0.0)
bfun=linear_interpolation((bx,by),bfake3.*masks[:,:,1];extrapolation_bc=0.0)


# Parameters for the vertical grid generation, adapted from https://getm.eu/files/GETM/doc/html/node14.html

du=5.001
dl=.001

DG=200
sk=sigma[1,1,2]
skm=sigma[1,1,1]
if iszero(du) && iszero(dl)
    function betaHB(sigma)
        return sigma
    end
    function invbetaHB(beta)
        return beta
    end
else
    function betaHB(sigma)
        #if du and dl are zero just return sigma
        return -(tanh((dl+du)*(1.0 - sigma) - dl)+tanh(dl))/(tanh(du)+tanh(dl))+1.0
    end
    function invbetaHB(beta)
    # test needed, not for the gridding but for data that might fall outside of the grid
        if beta>1.0 || beta<-1.0
            return beta
        end
    #if du and dl are zero just return beta
        return 1.0-(atanh(-(beta-1.0)*(tanh(du)+tanh(dl))-tanh(dl))+dl)/(dl+du)
    
    end
end
    
    
    
function alphaHB(x,y)
   
    alphanum=(betaHB(sk)-betaHB(skm))+DG/max(2.0,bfun(x,y))*(sk-skm)
    alphaden=(betaHB(sk)-betaHB(skm))+(sk-skm)
    alpha=max(min(alphanum/alphaden,1.0),0.0)
    return alpha
end
    
function sigmatoz(x,y,sigma)
    alpha=alphaHB(x,y)
    return  ((1-alpha)*betaHB(sigma)+alpha*sigma)*max(2.0,bfun(x,y))
end
# For more general cases the inverse will need some
# search of zeros. Needed to get fracional indexes and horizontal plots

function ztosigma(x,y,z)
    gamma=z/max(2.0,bfun(x,y))
    alpha=alphaHB(x,y)
    if alpha<0.001 
        return invbetaHB(gamma)
    end
    if alpha>0.999
        return gamma
    end
    guess=alpha*gamma+(1-alpha)*invbetaHB(gamma)
    # now improve by finding zero
    guess=find_zero(x->alpha*x+(1-alpha)*betaHB(x)-gamma, guess)
    return guess
end


In [ ]:
# Check inverse function
var(invbetaHB.(betaHB.(sigma))-sigma),var(sigma)



In [ ]:
zs=sigmatoz.(xs,ys,sigma)
# Check inverse function
testsigma=ztosigma.(xs,ys,zs)
var(testsigma.-sigma),var(sigma)

In [ ]:
# We already have the z positions of the sigma layers zs
# So now just recalculate the metrics:
pozc=deepcopy(pos)
for i = 1:sz[3]
        i0 = max(i - 1, 1)
        i1 = min(i + 1, sz[3])
        pozc[:,:,i].=(i1-i0)./(zs[:,:,i1].-zs[:,:,i0])
end



In [ ]:
# Now make the mask by using the zs positions of the grid and use the "real" topography for the mask:

bm=b.*mask_edit[:,:,1]
for k=1:size(zs)[3]
    masks[:,:,k].=zs[:,:,k].<bm[:,:] .&& mask_edit[:,:,1]
end

In [ ]:

pcolor(xs[:,jcut,:],-zs[:,jcut,:],sigma[:,jcut,:],shading="nearest"),colorbar()
xlim(16.5,19.5)

In [ ]:

pcolor(xs[:,jcut,:],-zs[:,jcut,:],zs[:,jcut,:],shading="nearest"),colorbar()
xlim(16.5,19.5)

### Vertical position in sigma space

In [ ]:
obssigma=ztosigma.(obslon,obslat,obsdepth)

### Get fractional indexes by working in sigma space:

In [ ]:
Igrid = localize_separable_grid((obslon,obslat,obssigma), masks, (xs,ys,sigma), iscyclic)

lenx = fill(LHOR,sz)  
leny = lenx./aspectratio
lenz = Lvert.(xs,ys,zs);
epsilon2 = eps2;

fi,s=DIVAndrun(masks,(pms,pns,pozc),(xs,ys,zs),(obslon,obslat,obsdepth),obsval.-obsmean,(lenx,leny,lenz),epsilon2;fracindex =Igrid);

In [ ]:
pcolor(xs[:,jcut,:],-zs[:,jcut,:],fi[:,jcut,:].+obsmean,shading="nearest"),colorbar()
#contour(xs[:,jcut,:],-zs[:,jcut,:],fi[:,jcut,:].+obsmean,30),colorbar()
plot(bx,-b[:,jcut])
xlim(16.5,19.5)
ylim(-1200,50)

## How to make a horizontal plot



In [ ]:
# Define depth for cut
zcut=zc[1,1,kcut]
# Create fake data and calculte sigma values on the constant z surface
sigmaonzlevel=ztosigma.(xs[:,:,1],ys[:,:,1],zcut.+ 0.0 .*ys[:,:,1])

# Get fractional indexes of the fake data and the observing operator
Igridcut=localize_separable_grid((xs[:,:,1],ys[:,:,1],sigmaonzlevel), masks, (xs,ys,sigma), )
Htoz, out, outbbox = sparse_interp(masks, Igridcut, iscyclic)
Htoz=Htoz*sparse_pack(masks)';

# Now observe the gridded field with that observation operator
zlevelsol=Htoz*statevector_pack(s.sv, (fi,))

# Put it in shape for plotting
soltoplot=reshape(zlevelsol,size(xs)[1:2])
# Need to apply mask because that is not included in the interpolation
soltoplot[zcut.>b].=NaN
soltoplot[mask_edit[:,:,1].==0].=NaN;

In [ ]:
pcolor(xs[:,:,1],ys[:,:,1],soltoplot.+obsmean,shading="nearest"),colorbar()
title("$zcut m")
gca().set_aspect(aspectratio)

In [ ]:
# Surface
pcolor(xs[:,:,1],ys[:,:,1],fi[:,:,1].+obsmean,shading="nearest"),colorbar()
gca().set_aspect(aspectratio)